In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from collections import Counter

# Import our input dataset
supply_chain_df = pd.read_csv('DataCoSupplyChainDataset.csv',encoding='ISO-8859-1')
supply_chain_df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [17]:
#Remove un-needed columns
supply_chain_clean_df = supply_chain_df.drop(columns=["Customer Email","Customer Fname","Category Id","Customer Lname","Customer Password","Customer City","Customer Id","Customer Street","Customer Zipcode",
                                                      "Latitude","Longitude","Department Id","Department Name","Order Customer Id","Order Id","Order Item Cardprod Id","Order Item Product Price","Order Item Discount",
                                                     "Product Card Id","Product Category Id","Product Description","Product Name","Product Price","Product Image","Order Zipcode","Sales per customer","Delivery Status",
                                                     "Order City","Order Item Discount Rate","Sales","Order Status","Product Status","order date (DateOrders)","shipping date (DateOrders)"])
supply_chain_clean_df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Late_delivery_risk,Category Name,Customer Country,Customer Segment,Customer State,Market,Order Country,Order Item Id,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Order Region,Order State,Shipping Mode
0,DEBIT,3,4,91.250000,0,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,Indonesia,180517,0.29,1,314.640015,91.250000,Southeast Asia,Java Occidental,Standard Class
1,TRANSFER,5,4,-249.089996,1,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,India,179254,-0.80,1,311.359985,-249.089996,South Asia,Rajastán,Standard Class
2,CASH,4,4,-247.779999,0,Sporting Goods,EE. UU.,Consumer,CA,Pacific Asia,India,179253,-0.80,1,309.720001,-247.779999,South Asia,Rajastán,Standard Class
3,DEBIT,3,4,22.860001,0,Sporting Goods,EE. UU.,Home Office,CA,Pacific Asia,Australia,179252,0.08,1,304.809998,22.860001,Oceania,Queensland,Standard Class
4,PAYMENT,2,4,134.210007,0,Sporting Goods,Puerto Rico,Corporate,PR,Pacific Asia,Australia,179251,0.45,1,298.250000,134.210007,Oceania,Queensland,Standard Class


In [18]:
data_types_df = pd.DataFrame(supply_chain_clean_df.dtypes)
data_types_df

,0
Type,object
Days for shipping (real),int64
Days for shipment (scheduled),int64
Benefit per order,float64
Late_delivery_risk,int64
Category Name,object
Customer Country,object
Customer Segment,object
Customer State,object
Market,object


In [19]:
supply_chain_clean_df = supply_chain_clean_df.dropna(axis='columns', how='all')
supply_chain_clean_df

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Late_delivery_risk,Category Name,Customer Country,Customer Segment,Customer State,Market,Order Country,Order Item Id,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Order Region,Order State,Shipping Mode
0,DEBIT,3,4,91.250000,0,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,Indonesia,180517,0.29,1,314.640015,91.250000,Southeast Asia,Java Occidental,Standard Class
1,TRANSFER,5,4,-249.089996,1,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,India,179254,-0.80,1,311.359985,-249.089996,South Asia,Rajastán,Standard Class
2,CASH,4,4,-247.779999,0,Sporting Goods,EE. UU.,Consumer,CA,Pacific Asia,India,179253,-0.80,1,309.720001,-247.779999,South Asia,Rajastán,Standard Class
3,DEBIT,3,4,22.860001,0,Sporting Goods,EE. UU.,Home Office,CA,Pacific Asia,Australia,179252,0.08,1,304.809998,22.860001,Oceania,Queensland,Standard Class
4,PAYMENT,2,4,134.210007,0,Sporting Goods,Puerto Rico,Corporate,PR,Pacific Asia,Australia,179251,0.45,1,298.250000,134.210007,Oceania,Queensland,Standard Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,0,Fishing,EE. UU.,Home Office,NY,Pacific Asia,China,65177,0.10,1,399.980011,40.000000,Eastern Asia,Shanghái,Standard Class
180515,DEBIT,3,2,-613.770019,1,Fishing,EE. UU.,Corporate,CA,Pacific Asia,Japón,65161,-1.55,1,395.980011,-613.770019,Eastern Asia,Osaka,Second Class
180516,TRANSFER,5,4,141.110001,1,Fishing,EE. UU.,Corporate,CT,Pacific Asia,Australia,65129,0.36,1,391.980011,141.110001,Oceania,Australia del Sur,Standard Class
180517,PAYMENT,3,4,186.229996,0,Fishing,Puerto Rico,Consumer,PR,Pacific Asia,Australia,65126,0.48,1,387.980011,186.229996,Oceania,Australia del Sur,Standard Class


In [20]:
# Remove Null columns
supply_chain_clean_df = supply_chain_clean_df.dropna()
supply_chain_clean_df

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Late_delivery_risk,Category Name,Customer Country,Customer Segment,Customer State,Market,Order Country,Order Item Id,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Order Region,Order State,Shipping Mode
0,DEBIT,3,4,91.250000,0,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,Indonesia,180517,0.29,1,314.640015,91.250000,Southeast Asia,Java Occidental,Standard Class
1,TRANSFER,5,4,-249.089996,1,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,India,179254,-0.80,1,311.359985,-249.089996,South Asia,Rajastán,Standard Class
2,CASH,4,4,-247.779999,0,Sporting Goods,EE. UU.,Consumer,CA,Pacific Asia,India,179253,-0.80,1,309.720001,-247.779999,South Asia,Rajastán,Standard Class
3,DEBIT,3,4,22.860001,0,Sporting Goods,EE. UU.,Home Office,CA,Pacific Asia,Australia,179252,0.08,1,304.809998,22.860001,Oceania,Queensland,Standard Class
4,PAYMENT,2,4,134.210007,0,Sporting Goods,Puerto Rico,Corporate,PR,Pacific Asia,Australia,179251,0.45,1,298.250000,134.210007,Oceania,Queensland,Standard Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,0,Fishing,EE. UU.,Home Office,NY,Pacific Asia,China,65177,0.10,1,399.980011,40.000000,Eastern Asia,Shanghái,Standard Class
180515,DEBIT,3,2,-613.770019,1,Fishing,EE. UU.,Corporate,CA,Pacific Asia,Japón,65161,-1.55,1,395.980011,-613.770019,Eastern Asia,Osaka,Second Class
180516,TRANSFER,5,4,141.110001,1,Fishing,EE. UU.,Corporate,CT,Pacific Asia,Australia,65129,0.36,1,391.980011,141.110001,Oceania,Australia del Sur,Standard Class
180517,PAYMENT,3,4,186.229996,0,Fishing,Puerto Rico,Consumer,PR,Pacific Asia,Australia,65126,0.48,1,387.980011,186.229996,Oceania,Australia del Sur,Standard Class


In [21]:
supply_chain_clean_10_df = supply_chain_clean_df.head(18000)


In [22]:
columns = supply_chain_clean_10_df.index
target = ["Late_delivery_risk"]

In [23]:
# Convert Late delivery risk
x = {1: 'sent late',0: 'sent on time'}  
# Change tarrget to difference betweenn days for shipping real and days for shipment scheduled
supply_chain_clean_10_df["Late_delivery_risk"] = supply_chain_clean_10_df["Late_delivery_risk"].replace(x)
supply_chain_clean_10_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Late_delivery_risk,Category Name,Customer Country,Customer Segment,Customer State,Market,Order Country,Order Item Id,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Order Region,Order State,Shipping Mode
0,DEBIT,3,4,91.250000,sent on time,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,Indonesia,180517,0.29,1,314.640015,91.250000,Southeast Asia,Java Occidental,Standard Class
1,TRANSFER,5,4,-249.089996,sent late,Sporting Goods,Puerto Rico,Consumer,PR,Pacific Asia,India,179254,-0.80,1,311.359985,-249.089996,South Asia,Rajastán,Standard Class
2,CASH,4,4,-247.779999,sent on time,Sporting Goods,EE. UU.,Consumer,CA,Pacific Asia,India,179253,-0.80,1,309.720001,-247.779999,South Asia,Rajastán,Standard Class
3,DEBIT,3,4,22.860001,sent on time,Sporting Goods,EE. UU.,Home Office,CA,Pacific Asia,Australia,179252,0.08,1,304.809998,22.860001,Oceania,Queensland,Standard Class
4,PAYMENT,2,4,134.210007,sent on time,Sporting Goods,Puerto Rico,Corporate,PR,Pacific Asia,Australia,179251,0.45,1,298.250000,134.210007,Oceania,Queensland,Standard Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,CASH,3,2,30.260000,sent late,Men's Footwear,EE. UU.,Corporate,CA,Europe,Francia,46913,0.24,1,126.089996,30.260000,Western Europe,Languedoc-Rosellón-Mediodía-Pirineos,Second Class
17996,CASH,4,2,30.170000,sent late,Children's Clothing,EE. UU.,Corporate,CA,Europe,Italia,173763,0.09,1,342.820007,30.170000,Southern Europe,Emilia-Romaña,Second Class
17997,CASH,5,2,20.910000,sent late,Cleats,EE. UU.,Corporate,CO,Europe,Bosnia y Herzegovina,120928,0.36,1,57.590000,20.910000,Southern Europe,Federación de Bosnia y Herzegovina,Second Class
17998,CASH,4,2,16.530001,sent late,Cleats,EE. UU.,Corporate,CA,Europe,Francia,169860,0.29,1,56.990002,16.530001,Western Europe,Borgoña-Franco Condado,Second Class


In [24]:
# Create our features
X = supply_chain_clean_10_df.drop("Late_delivery_risk", axis=1)

X = pd.get_dummies(X)

# Create our target
y = supply_chain_clean_10_df["Late_delivery_risk"]

In [25]:
X.describe()

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Order Item Id,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Type_CASH,Type_DEBIT,...,Order State_Zhejiang,Order State_Zhytómyr,Order State_Zulia,Order State_Zúrich,Order State_iauliai,Order State_Équateur,Shipping Mode_First Class,Shipping Mode_Same Day,Shipping Mode_Second Class,Shipping Mode_Standard Class
count,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,...,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000,18000.000000
mean,3.613389,3.050667,23.018066,89795.299222,0.124127,2.634444,187.855219,23.018066,0.208722,0.402722,...,0.001000,0.000056,0.000889,0.000611,0.000111,0.000222,0.123556,0.038667,0.212000,0.625778
std,1.580877,1.295193,111.730532,52568.452466,0.462575,1.599743,128.003787,111.730532,0.406407,0.490459,...,0.031608,0.007454,0.029802,0.024714,0.010541,0.014906,0.329083,0.192805,0.408736,0.483935
min,0.000000,0.000000,-3366.000000,35.000000,-2.750000,1.000000,8.470000,-3366.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,7.660000,44573.500000,0.080000,1.000000,106.209999,7.660000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,32.279999,86613.000000,0.270000,2.000000,163.979996,32.279999,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,5.000000,4.000000,67.489998,134234.000000,0.360000,4.000000,251.984997,67.489998,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,6.000000,4.000000,684.000000,180519.000000,0.500000,5.000000,1759.989990,684.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
# Check the balance of our target values
y.value_counts()

sent late       9727
sent on time    8273
Name: Late_delivery_risk, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [28]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'sent late': 7211, 'sent on time': 7211})

In [29]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [30]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9677219024052515